## Installation

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.3 MB/s 
     |████████████████████████████████| 365 kB 61.3 MB/s 
     |████████████████████████████████| 6.6 MB 416 kB/s 
     |████████████████████████████████| 596 kB 15.6 MB/s 
     |████████████████████████████████| 101 kB 6.3 MB/s 
     |████████████████████████████████| 115 kB 56.4 MB/s 
     |████████████████████████████████| 212 kB 4.5 MB/s 
     |████████████████████████████████| 141 kB 64.6 MB/s 
     |████████████████████████████████| 127 kB 50.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Libraries

In [ ]:
import json

from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, default_data_collator
import transformers
import pandas as pd
from tqdm import tqdm

## Языковая Модель GPT-2

In [ ]:
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT - огромные модели, размеры большого варианта модели приближаются к миллиарду параметров.

In [ ]:
total_params = 0
for p in model.parameters():
  total_params += torch.prod(torch.tensor(p.shape))

print(f'Количество параметров сети: {total_params}')

Количество параметров сети: 125231616


In [ ]:
device = torch.device('cuda')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

## Dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df = pd.read_csv("result.csv")

In [ ]:
# df = df[::40]

In [ ]:
texts = df["all_data"].values

In [ ]:
train_size = int(texts.shape[0] * 0.7)

In [ ]:
with open('dataset.txt', 'w') as fp:
  fp.write("\n".join(texts[:train_size]))


with open('dataset_test.txt', 'w') as fp:
  fp.write("\n".join(texts[train_size + 1:]))

In [ ]:
with open('dataset.json', 'w') as fp:
  with open('dataset.txt') as fpt:
    texts = fpt.read().split('\n')
  for text in texts:
    fp.write(json.dumps({'input': text}) + '\n')


with open('dataset_test.json', 'w') as fp:
  with open('dataset_test.txt') as fpt:
    texts = fpt.read().split('\n')
  for text in texts:
    fp.write(json.dumps({'input': text}) + '\n')


In [ ]:
raw_dataset = load_dataset('json', data_files={'train': 'dataset.json', 'test': 'dataset_test.json'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b00634fa56d80742/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(example):
  return tokenizer(example['input'], return_tensors='pt')

def add_labels(example):
  example['labels'] = example['input_ids']
  return example
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
columns_to_remove = raw_dataset["train"].column_names
ds = raw_dataset.map(tokenize_function,  remove_columns=columns_to_remove, load_from_cache_file=False).map(add_labels, load_from_cache_file=False)
ds['train'][0]

  0%|          | 0/479 [00:00<?, ?ex/s]

  0%|          | 0/160 [00:00<?, ?ex/s]

  0%|          | 0/479 [00:00<?, ?ex/s]

  0%|          | 0/160 [00:00<?, ?ex/s]

{'attention_mask': [[1, 1]],
 'input_ids': [[15078, 30]],
 'labels': [[15078, 30]]}

In [ ]:
tokenizer.pad_token_id = 0
tokenizer.pad_token = '<pad>'
tokenizer.pad_token = tokenizer.get_vocab()


dl = DataLoader(ds['train'], batch_size=1, collate_fn=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))
dl_test = DataLoader(ds['test'], batch_size=1, collate_fn=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [ ]:
next(iter(dl_test))

{'input_ids': tensor([[[ 3290,    30,   677,  6648,  7992,  1051,    16, 25085, 34209,   334,
           2286, 27815,   309, 42409,   512,    16,   309, 25382,   289,   309,
           8440,   291,    17, 10202,   554,   609,     5]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1]]]), 'labels': tensor([[[ 3290,    30,   677,  6648,  7992,  1051,    16, 25085, 34209,   334,
           2286, 27815,   309, 42409,   512,    16,   309, 25382,   289,   309,
           8440,   291,    17, 10202,   554,   609,     5]]])}

### Дообучение модели

Для дообучения модели нам нужны тексты. Для примера создадим маленький датасет.

In [ ]:
lr = 1e-6
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
len(df)

535

In [ ]:
epochs = 32

for epoch in tqdm(range(epochs)):
  total_loss = 0
  for batch in dl:
    optimizer.zero_grad()
    output = model(**batch.to(device))
    output.loss.backward()
    optimizer.step()
    total_loss += output.loss.item()

  # Оцениваем loss на тестовой выборке
  # Экономим память чтобы не хранить информацию для градиентов
  with torch.no_grad():
    total_loss_test = 0
    for batch in dl_test:
      output = model(**batch.to(device))
      total_loss_test += output.loss.item()
  
  print(f'Mean loss on epoch {epoch}: train {total_loss / len(dl):.3f} test {total_loss_test / len(dl_test):.3f}')

  3%|▎         | 1/32 [01:20<41:24, 80.14s/it]

Mean loss on epoch 0: train 0.478 test 9.027


  6%|▋         | 2/32 [02:40<40:04, 80.15s/it]

Mean loss on epoch 1: train 0.400 test 9.036


  9%|▉         | 3/32 [04:00<38:48, 80.30s/it]

...

Mean loss on epoch 29: train 0.223 test 9.969


 97%|█████████▋| 31/32 [41:14<01:20, 80.26s/it]

Mean loss on epoch 30: train 0.223 test 9.983


100%|██████████| 32/32 [42:35<00:00, 79.85s/it]

Mean loss on epoch 31: train 0.222 test 9.981


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
d2 = {}  
for l in 'вечер, утро, перерыв, работа, еда, спорт'.split(', '):
  for i in range(30):
    out = model.generate(**tokenizer('работа:', return_tensors='pt').to(device), return_dict_in_generate=True, output_scores=True, do_sample=True, max_length=30,  pad_token_id=tokenizer.eos_token_id)
    txt = tokenizer.batch_decode(out.sequences)[0]
    # for j in '?!.':
    try:
      d2[l].append(txt)
    except KeyError:
      d2[l] = [txt]

      

In [ ]:
d

{'вечер': ['работа:Случайностей не бывает! Что в ней проку? Скажите-ка мне, кем вы будете, если лишить вас совпадению?          ',
  'работа:Что бы потом с нами ни случилось, это был великий день и великий час! В великий час мы стали еще больней, назло!  Которого мы не имеем?   великий день',
  'работа:Время делает шаг… вторник убивает среда… вторник убивает среда…  вторгается гнет, а не дар!  Четверть может, кому-то секрет… Вовочка, кому-то',
  'работа:В конце концов, именно такой человек и нужен… Который сам приходит, когда необходим… Которого никогда не нужно просить…  Которого никогда не нужно просить…  Которого никогда не нужно',
  'работа:Случайно все, но не случайно лишь последствие… учайно все, последствие — мачеха логики…  зачем-то случайность…  следствия, причины, следствия',
  'работа:<…> всякому деревцу родная земля слаще <…> <…> всё растёт, растёт… растёт…  растёт… растёт…  растёт… чтобы земля земля, нужно бы день и великий час! Вот',
  'работа:Найти лучшего друга — самая 

In [ ]:
out = model.generate(**tokenizer('работа:', return_tensors='pt').to(device), return_dict_in_generate=True, output_scores=True, do_sample=True, max_length=40)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
out.sequences

tensor([[ 3290,    30, 19563,  3991,  5406,    16,  1029,  1494,  1904,  1590,
         21899,   309, 27941, 19600,  6566,   482,  7385,  2974,   312,   303,
           871,    16,   685,  4508, 13426, 17542,     5,   889,   492,   289,
         12387,  1119,   289, 12387,  1808,     5,  1397,   492,  1574,   757]],
       device='cuda:0')

In [ ]:
tokenizer.batch_decode(out.sequences)[0]

'работа:Единственным человеком, который всегда хорошо смотрелся на заднем сиденье четырехместного кабриолета, был Адольф Гитлер! Это он и великий день и великий час! Что он такой мог'

In [ ]:
import json
with open('data.json', 'w') as f:
  json.dump(d2, f)